In [1]:
import numpy as np
import sys
sys.path.insert(1,'./build')
import biest_call
import time

In [2]:
a = 0.66 # np.random.random() * 0.2 + 0.8
b_single = [0.66, 0.77, 0.88] # np.random.random() * 0.2 + 0.8
b_double = [0.66, 0.77] # np.random.random() * 0.2 + 0.8

In [3]:
%%time
test_example_single = []
for i in range(len(b_single)):
    test_example_single.append(biest_call.test(True, a, b_single[i]))
test_example_double = []
for i in range(len(b_double)):
    test_example_double.append(biest_call.test(False, a, b_double[i]))
print('The BIEST single-layer example evaluates to', test_example_single)
print('The BIEST double-layer example evaluates to', test_example_double)

IOStream.flush timed out


The BIEST single-layer example evaluates to [0.36017343489628273, 0.3601734348962832, 0.3601734348962813]
The BIEST double-layer example evaluates to [0.48895257587060564, 0.48895257587060736]
CPU times: user 2h 23min 48s, sys: 17.1 s, total: 2h 24min 6s
Wall time: 20min 26s


IOStream.flush timed out


In [8]:
# Testing if BIEST is called correctly
nfp = 1
Nt = 70
Np = 20
DIM = 3
gamma = np.zeros((Nt, Np, DIM))
func_in_single = np.zeros((Nt, Np, len(b_single)))
func_in_double = np.zeros((Nt, Np, len(b_double)))
for k in range(len(b_single)):
    for i in np.arange(Nt):
        for j in np.arange(Np):
            b = b_single[k]
            phi = 2 * np.pi * i / Nt;
            theta = 2 * np.pi * j / Np;
            R = 1 + 0.25 * np.cos(theta);
            x = R * np.cos(phi);
            y = R * a * np.sin(phi);
            z = 0.25 * np.sin(theta);
            gamma[i, j, 0] = x;
            gamma[i, j, 1] = y;
            gamma[i, j, 2] = z;
            func_in_single[i, j, k] = x + y + b * z;
            
for k in range(len(b_double)):
    for i in np.arange(Nt):
        for j in np.arange(Np):
            b = b_double[k]
            phi = 2 * np.pi * i / Nt;
            theta = 2 * np.pi * j / Np;
            R = 1 + 0.25 * np.cos(theta);
            x = R * np.cos(phi);
            y = R * a * np.sin(phi);
            z = 0.25 * np.sin(theta);
            gamma[i, j, 0] = x;
            gamma[i, j, 1] = y;
            gamma[i, j, 2] = z;
            func_in_double[i, j, k] = x + y + b * z;

In [9]:
func_in_double[10, 10]

array([0.85462394, 0.85462394])

In [ ]:
time1 = time.time()
test_both = np.zeros((len(b_single) + len(b_double)), dtype=np.float64)
biest_call.integrate_multi(
    gamma, # xt::pyarray<double> &gamma,
    func_in_single, # xt::pyarray<double> &func_in_single,
    func_in_double, # xt::pyarray<double> &func_in_double,
    test_both, # xt::pyarray<double> &result,
    10, # int digits,
    1, # int nfp
)
time2 = time.time()
print('Both eval time:', time2 - time1)
print('Both result:', test_both)

time1 = time.time()
test_neither = np.zeros((0), dtype=np.float64)
biest_call.integrate_multi(
    gamma, # xt::pyarray<double> &gamma,
    None, # xt::pyarray<double> &func_in_single,
    None, # xt::pyarray<double> &func_in_double,
    test_neither, # xt::pyarray<double> &result,
    10, # int digits,
    1, # int nfp
)
time2 = time.time()
print('Neither eval time:', time2 - time1)
print('Neither result:', test_neither)

In [ ]:
# Testing the behavior of integrate_multi, and the option to 
# skip either arguments to save time
time1 = time.time()
test_single_only = np.zeros((1))
biest_call.integrate_multi(
    gamma, # xt::pyarray<double> &gamma,
    func_in_single, # xt::pyarray<double> &func_in_single,
    None, # xt::pyarray<double> &func_ia_singlele,
    test_single_only, # xt::pyarray<double> &result,
    10, # int digits,
    1, # int nfp
)
time2 = time.time()
print('Single-layer only eval time:', time2 - time1)
print('Single-layer only result:', test_single_only)

time1 = time.time()
test_double_only = np.zeros((1))
biest_call.integrate_multi(
    gamma, # xt::pyarray<double> &gamma,
    None, # xt::pyarray<double> &func_in_single,
    func_in_double, # xt::pyarray<double> &func_in_double,
    test_double_only, # xt::pyarray<double> &result,
    10, # int digits,
    1, # int nfp
)
time2 = time.time()
print('Double-layer only eval time:', time2 - time1)
print('Double-layer only result:', test_double_only)


In [5]:
test_both

array([0.36017343, 0.48895258])